In [2]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import noisereduce as nr
from keras.models import model_from_json
from sklearn.preprocessing import LabelEncoder
import IPython
import os
import csv

In [4]:
# Some Utils

# Plot audio with zoomed in y axis
def plotAudio(output):
    fig, ax = plt.subplots(nrows=1,ncols=1, figsize=(20,10))
    plt.plot(output, color='blue')
    ax.set_xlim((0, len(output)))
    ax.margins(2, -0.1)
    plt.show()

# Plot audio
def plotAudio2(output):
    fig, ax = plt.subplots(nrows=1,ncols=1, figsize=(20,4))
    plt.plot(output, color='blue')
    ax.set_xlim((0, len(output)))
    plt.show()

# Split a given long audio file on silent parts.
# Accepts audio numpy array audio_data, window length w and hop length h, threshold_level, tolerence
# threshold_level: Silence threshold
# Higher tolence to prevent small silence parts from splitting the audio.
# Returns array containing arrays of [start, end] points of resulting audio clips
def split_audio(audio_data, w, h, threshold_level, tolerence=10):
    split_map = []
    start = 0
    data = np.abs(audio_data)
    threshold = threshold_level*np.mean(data[:20000])
    inside_sound = False
    near = 0
    for i in range(0,len(data)-w, h):
        win_mean = np.mean(data[i:i+w])
        if(win_mean>threshold and not(inside_sound)):
#             print(i, 'inside sound')
            inside_sound = True
            start = i
        if(win_mean<=threshold and inside_sound and near>tolerence):
#             print(i, 'outside sound')
            inside_sound = False
            near = 0
            split_map.append([start, i])
        if(inside_sound and win_mean<=threshold):
            near += 1
    return split_map

def minMaxNormalize(arr):
    mn = np.min(arr)
    mx = np.max(arr)
    return (arr-mn)/(mx-mn)

def predictSound(X):
    stfts = np.abs(librosa.stft(X, n_fft=512, hop_length=256, win_length=512))
    stfts = np.mean(stfts,axis=1)
#     stfts = minMaxNormalize(stfts)
    result = model.predict(np.array([stfts]))
    predictions = [np.argmax(y) for y in result]
    return lb.inverse_transform([predictions[0]])[0]

def get_win_mean(audio_data, w, h, threshold_level):
    data = np.abs(audio_data)
    win_mean = np.zeros(shape = raw_audio.shape, dtype = raw_audio.dtype)
    threshold = threshold_level*np.mean(data[:20000])
    print('np.mean(data[:20000]) = ', np.mean(data[:20000]))
    print('threshold = ', threshold)
    for i in range(0,len(data)-w, h):
        win_mean[i] = np.mean(data[i:i+w])
    return win_mean

In [ ]:
# create testing data

activities = ['Calling', 'Clapping', 'Drinking', 'Eating', 'Entering',
              'Exiting', 'Falling', 'LyingDown', 'OpeningPillContainer',
              'PickingObject', 'Reading', 'SitStill', 'Sitting', 'Sleeping',
              'StandUp', 'Sweeping', 'UseLaptop', 'UsingPhone', 'WakeUp', 'Walking',
              'WashingHand', 'WatchingTV', 'WaterPouring', 'Writing']
    
subjects = ['s01', 's02', 's03', 's04', 's05', 's06', 's07', 's08', 's09',
            's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17']

test_subjects = ['s05', 's17']

mergedActivities = ['Drinking', 'Eating', 'LyingDown', 'OpeningPillContainer', 
                      'PickingObject', 'Reading', 'SitStill', 'Sitting', 'Sleeping', 
                      'StandUp', 'UseLaptop', 'UsingPhone', 'WakeUp', 'Walking', 
                      'WaterPouring', 'Writing']

specificActivities = ['Calling', 'Clapping', 'Falling', 'Sweeping', 'WashingHand', 'WatchingTV']

enteringExiting = ['Entering', 'Exiting']


def get_model_label(activity):
    model_label = ''
    if activity in specificActivities:
        model_label = activity
    elif activity in enteringExiting:
        model_label = 'enteringExiting'
    elif activity in mergedActivities:
        model_label = 'other' 
    
    return model_label
                
    
for activity in activities:
    for subject in test_subjects:
        innerDir = subject + "/" + activity
        for file in os.listdir("Dataset_audio/" + innerDir):
            if(file.endswith(".wav")):
                model_label = get_model_label(activity)
                save_prediction("Dataset_audio/" + innerDir + "/" + file, file, model_label, subject)
                print(subject,activity,file)